# SimCLR
PyTorch implementation of SimCLR: A Simple Framework for Contrastive Learning of Visual Representations by T. Chen et al. With support for the LARS (Layer-wise Adaptive Rate Scaling) optimizer.

[Link to paper](https://arxiv.org/pdf/2002.05709.pdf)


## Setup the repository

In [1]:
# !pip install pretrainedmodels

# !git clone https://github.com/spijkervet/SimCLR.git
# %cd SimCLR
# !wget https://github.com/Spijkervet/SimCLR/releases/download/1.2/checkpoint_100.tar
# !sh setup.sh || python3 -m pip install -r requirements.txt || exit 1
# !pip install  pyyaml --upgrade

In [2]:
# !git clone https://github.com/Kabongosalomon/SimCLR-1.git
%cd SimCLR-1
# !wget https://github.com/Spijkervet/SimCLR/releases/download/1.2/checkpoint_100.tar
# !sh setup.sh || python3 -m pip install -r requirements.txt || exit 1
# !pip install  pyyaml --upgrade

/root/cassava_disease_classification/salomon_exp/SimCLR-1


In [3]:
# !pip install ipdb
import ipdb

import torch
import torchvision
import torchvision.transforms as transforms
import argparse

# from experiment import ex
from model import load_model
from utils import post_config_hook

from modules import LogisticRegression

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import glob
import numpy as np

from torchvision import transforms, datasets, models


from PIL import Image

# Part 1:
## SimCLR pre-training

In [4]:
# whether to use a TPU or not (set in Runtime -> Change Runtime Type)
use_tpu = False

#### Install PyTorch/XLA

In [5]:
if use_tpu:
    VERSION = "20200220" #@param ["20200220","nightly", "xrt==1.15.0"]
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $VERSION

In [6]:
import os
import torch

if use_tpu:
    # imports the torch_xla package for TPU support
    import torch_xla
    import torch_xla.core.xla_model as xm
    dev = xm.xla_device()
    print(dev)

import torchvision
import argparse

from torch.utils.tensorboard import SummaryWriter

apex = False
try:
    from apex import amp
    apex = True
except ImportError:
    print(
        "Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training"
    )

from model import load_model, save_model
from modules import NT_Xent
from modules.transformations import TransformsSimCLR
from utils import post_config_hook

Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training


### Load arguments from `config/config.yaml`

In [7]:
from pprint import pprint
from utils.yaml_config_hook import yaml_config_hook

config = yaml_config_hook("./config/config.yaml")
args = argparse.Namespace(**config)

if use_tpu:
    args.device = dev
else:
    args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

args.out_dir = "logs"
if not os.path.exists("logs"):
    os.makedirs("logs")

In [8]:
pprint(vars(args))

{'batch_size': 128,
 'dataset': 'CIFAR10',
 'device': device(type='cuda', index=0),
 'epoch_num': 100,
 'epochs': 100,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 500,
 'model_path': 'logs/0',
 'normalize': True,
 'optimizer': 'Adam',
 'out_dir': 'logs',
 'pretrain': True,
 'projection_dim': 64,
 'resnet': 'resnet50',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [9]:
# 128/2

In [10]:
### override any configuration parameters here, e.g. to adjust for use on GPUs on the Colab platform:
args.batch_size = 64
args.resnet = "resnet18" # se_resnet50, resnet18, se_resnext101_32x4d, resnet50
pprint(vars(args))

{'batch_size': 64,
 'dataset': 'CIFAR10',
 'device': device(type='cuda', index=0),
 'epoch_num': 100,
 'epochs': 100,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 500,
 'model_path': 'logs/0',
 'normalize': True,
 'optimizer': 'Adam',
 'out_dir': 'logs',
 'pretrain': True,
 'projection_dim': 64,
 'resnet': 'resnet18',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [11]:
args.dataset = 'casava'

In [12]:
# args.epochs = 100
# args.epoch_num = 100

# args.projection_dim = 64

# args.logistic_epochs = 100

# args.optimizer = "LARS" #"Adam"

### Load dataset into train loader

In [13]:
data_path = "../data/train/train"
test_path = "../data/test/test"
extraimage_path = "../data/extraimages/extraimages"

In [14]:
print('Train set:')
class_distrbution = {}
for cls in os.listdir(data_path):
    print('{}:{}'.format(cls, len(os.listdir(os.path.join(data_path, cls)))))
    class_distrbution[cls] =  len(os.listdir(os.path.join(data_path, cls)))
im = Image.open(data_path+'/cgm/train-cgm-738.jpg')
print(im.size)
class_distrbution

Train set:
cbb:466
healthy:316
cbsd:1443
cmd:2658
cgm:773
(500, 500)


{'cbb': 466, 'cbsd': 1443, 'cgm': 773, 'cmd': 2658, 'healthy': 316}

In [15]:
class CassavaDataset(Dataset):
    def __init__(self, path, size, s=1, mutation = False):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.mutation = mutation
        color_jitter = torchvision.transforms.ColorJitter(
            0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s
        )
        
        self.train_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.RandomResizedCrop(size),
                torchvision.transforms.RandomHorizontalFlip(),  # with 0.5 probability
                torchvision.transforms.RandomApply([color_jitter], p=0.8),
                torchvision.transforms.RandomGrayscale(p=0.2),
                torchvision.transforms.ToTensor(),
            ]
        )
        
        self.test_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize((size, size)),
                torchvision.transforms.ToTensor()
            ]
        )
        

        files = []
        class_names = {}
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])

                name = str(i)+'-'+className
                if name not in class_names:
                    class_names[name] = 1
                else:
                    class_names[name] += 1
        self.file_list = files
        files = None
    
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        image = Image.open(fileName)

        if self.mutation:
            image1 = self.train_transform(image)
            image2 = self.train_transform(image)
            
            sample = [[image1, image2], classCategory]
        else:
            
            image = self.test_transform(image)
            sample = [image, classCategory]

        return sample

In [16]:
size = 448

train_data = CassavaDataset(data_path, size, s=1, mutation = False)

test_data = CassavaDataset(test_path, size, s=1, mutation = False)

extraimage_data = CassavaDataset(extraimage_path, size, s=1, mutation = True)

#######################################################################
validation_split = 0.0
shuffle_dataset = True
# random_seed= 42 #42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
#     np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]
########################################################################

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

batch_size_train = args.batch_size# 125
batch_size_eval = args.batch_size#250
n_workers = 2

# train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size_train,
#                                              sampler = train_sampler, num_workers = n_workers)

# valid_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size_eval,
#                                              sampler = valid_sampler, num_workers = n_workers)

unlabeled_loader = torch.utils.data.DataLoader(extraimage_data, batch_size = batch_size_eval, 
                                              shuffle =shuffle_dataset, num_workers = n_workers)


In [17]:
# # number of subprocesses to use for data loading
# num_workers = 6
# # how many samples per batch to load
# batch_size = 125

# train_data = datasets.ImageFolder(data_path, transform=train_transforms)
# test_data = datasets.ImageFolder(test_path, transform=test_transforms)
# extraimage_data = datasets.ImageFolder(extraimage_path, transform=train_transforms)

# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
#                                            num_workers=num_workers, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
#                                           num_workers=num_workers)

# unlabeled_loader = torch.utils.data.DataLoader(extraimage_data, batch_size=batch_size,
#                                                num_workers=num_workers) # to make batch_size work, I had to moove all the unlabeled data in a 0 folder

In [18]:
# next(unlabeled_loader.__iter__())

In [19]:
# len(next(unlabeled_loader.__iter__())[1])

In [20]:
# root = "./datasets"

# train_sampler = None

# if args.dataset == "STL10":
#     train_dataset = torchvision.datasets.STL10(
#         root, split="unlabeled", download=True, transform=TransformsSimCLR(size=96)
#     )
# elif args.dataset == "CIFAR10":
#     train_dataset = torchvision.datasets.CIFAR10(
#         root, download=True, transform=TransformsSimCLR(size=32)
#     )
# else:
#     raise NotImplementedError

# train_loader = torch.utils.data.DataLoader(
#     train_dataset,
#     batch_size=args.batch_size,
#     shuffle=(train_sampler is None),
#     drop_last=True,
#     num_workers=args.workers,
#     sampler=train_sampler,
# )

### Load the SimCLR model, optimizer and learning rate scheduler

In [21]:
pprint(vars(args))

{'batch_size': 64,
 'dataset': 'casava',
 'device': device(type='cuda', index=0),
 'epoch_num': 100,
 'epochs': 100,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 500,
 'model_path': 'logs/0',
 'normalize': True,
 'optimizer': 'Adam',
 'out_dir': 'logs',
 'pretrain': True,
 'projection_dim': 64,
 'resnet': 'resnet18',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [22]:
# args.projection_dim = 125

In [23]:
model, optimizer, scheduler = load_model(args, unlabeled_loader, reload_model=False)

In [24]:
model

SimCLR(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [25]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0002
    weight_decay: 0
)

In [26]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0002
    weight_decay: 0
)

In [27]:
scheduler

### Setup TensorBoard for logging experiments

In [28]:
tb_dir = os.path.join(args.out_dir, "colab")
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
writer = SummaryWriter(log_dir=tb_dir)

### Create the mask that will remove correlated samples from the negative examples

### Initialize the criterion (NT-Xent loss)

In [29]:
args.batch_size

64

In [30]:
criterion = NT_Xent(args.batch_size, args.temperature, args.device)

In [31]:
criterion

NT_Xent(
  (criterion): CrossEntropyLoss()
  (similarity_f): CosineSimilarity()
)

### Start training

In [32]:
def train(args, train_loader, model, criterion, optimizer, writer):
    loss_epoch = 0
    for step, ((x_i, x_j), _) in enumerate(train_loader):

        if x_i.shape[0] != args.batch_size:
            continue
        optimizer.zero_grad()
        x_i = x_i.to(args.device)
        x_j = x_j.to(args.device)

        # positive pair, with encoding
        h_i, z_i = model(x_i)
        h_j, z_j = model(x_j)

#         ipdb.set_trace()
        loss = criterion(z_i, z_j)

        if apex and args.fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()

        optimizer.step()

        if step % 50 == 0:
            print(f"Step [{step}/{len(train_loader)}]\t Loss: {loss.item()}")

        writer.add_scalar("Loss/train_epoch", loss.item(), args.global_step)
        loss_epoch += loss.item()
        args.global_step += 1

    return loss_epoch

In [ ]:
import pdb, traceback, sys

if __name__ == '__main__':
    try:
        
        args.global_step = 0
        args.current_epoch = 0
        for epoch in range(args.start_epoch, args.epochs):
            lr = optimizer.param_groups[0]['lr']
        #     ipdb.set_trace()
            loss_epoch = train(args, unlabeled_loader, model, criterion, optimizer, writer)

            if scheduler:
                scheduler.step()

            if epoch % 100 == 0:
                save_model(args, model, optimizer)

            writer.add_scalar("Loss/train", loss_epoch / len(unlabeled_loader), epoch)
            writer.add_scalar("Misc/learning_rate", lr, epoch)
            print(
                f"Epoch [{epoch}/{args.epochs}]\t Loss: {loss_epoch / len(unlabeled_loader)}\t lr: {round(lr, 5)}"
            )
            args.current_epoch += 1

        ## end training
        save_model(args, model, optimizer)
    except:
        extype, value, tb = sys.exc_info()
        traceback.print_exc()
        pdb.post_mortem(tb)

Step [0/197]	 Loss: 4.825965881347656
Step [50/197]	 Loss: 4.679616451263428
Step [100/197]	 Loss: 4.63761043548584
Step [150/197]	 Loss: 4.709129333496094
Epoch [0/100]	 Loss: 4.6434118445149535	 lr: 0.0002
Step [0/197]	 Loss: 4.479612827301025
Step [50/197]	 Loss: 4.378774166107178
Step [100/197]	 Loss: 4.248110294342041


In [31]:
# import pdb, traceback, sys

# def bombs():
#     a = []
#     print (a[0])

# if __name__ == '__main__':
#     try:
#         bombs()
#     except:
#         extype, value, tb = sys.exc_info()
#         traceback.print_exc()
#         pdb.post_mortem(tb)

## Download last checkpoint to local drive (replace `100` with `args.epochs`)

In [39]:
# from google.colab import files
# files.download('./logs/checkpoint_'+args.epochs.tar) # checkpoint_100.tar

# Part 2:
## Linear evaluation using logistic regression, using weights from frozen, pre-trained SimCLR model

In [ ]:
def train(args, loader, simclr_model, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    for step, (x, y) in enumerate(loader):
        optimizer.zero_grad()

        x = x.to(args.device)
        y = y.to(args.device)

        # get encoding
        with torch.no_grad():
            h, z = simclr_model(x)
            # h = 512
            # z = 64

        output = model(h)
        loss = criterion(output, y)

        predicted = output.argmax(1)
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc

        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()
        if step % 100 == 0:
            print(f"Step [{step}/{len(loader)}]\t Loss: {loss.item()}\t Accuracy: {acc}")

    return loss_epoch, accuracy_epoch

In [ ]:
def test(args, loader, simclr_model, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    model.eval()
    for step, (x, y) in enumerate(loader):
        model.zero_grad()

        x = x.to(args.device)
        y = y.to(args.device)

        # get encoding
        with torch.no_grad():
            h, z = simclr_model(x)
            # h = 512
            # z = 64

        output = model(h)
        loss = criterion(output, y)

        predicted = output.argmax(1)
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc

        loss_epoch += loss.item()


    return loss_epoch, accuracy_epoch

In [32]:
# from pprint import pprint
# from utils.yaml_config_hook import yaml_config_hook

# config = yaml_config_hook("./config/config.yaml")
# pprint(config)
# args = argparse.Namespace(**config)

# if use_tpu:
#     args.device = dev
# else:
#     args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

{'batch_size': 128,
 'dataset': 'CIFAR10',
 'epoch_num': 100,
 'epochs': 100,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 500,
 'model_path': 'logs/0',
 'normalize': True,
 'optimizer': 'Adam',
 'pretrain': True,
 'projection_dim': 64,
 'resnet': 'resnet50',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [33]:
# args.batch_size = 32
# args.resnet = "se_resnext101_32x4d" # 
# args.model_path = "logs"
# args.epoch_num = 20 # 100
# args.projection_dim = 32

# args.logistic_epochs = 100

### Load dataset into train/test dataloaders

In [35]:
# root = "./datasets"
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# if args.dataset == "STL10":
#     train_dataset = torchvision.datasets.STL10(
#         root, split="train", download=True, transform=torchvision.transforms.ToTensor()
#     )
#     test_dataset = torchvision.datasets.STL10(
#         root, split="test", download=True, transform=torchvision.transforms.ToTensor()
#     )
# elif args.dataset == "CIFAR10":
#     train_dataset = torchvision.datasets.CIFAR10(
#         root, train=True, download=True, transform=transform
#     )
#     test_dataset = torchvision.datasets.CIFAR10(
#         root, train=False, download=True, transform=transform
#     )
# else:
#     raise NotImplementedError

# train_loader = torch.utils.data.DataLoader(
#     train_dataset,
#     batch_size=args.logistic_batch_size,
#     shuffle=True,
#     drop_last=True,
#     num_workers=args.workers,
# )

# test_loader = torch.utils.data.DataLoader(
#     test_dataset,
#     batch_size=args.logistic_batch_size,
#     shuffle=False,
#     drop_last=True,
#     num_workers=args.workers,
# )

### Load SimCLR model and load model weights

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size_train,
                                             sampler = train_sampler, num_workers = n_workers)

valid_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size_eval,
                                             sampler = valid_sampler, num_workers = n_workers)

In [ ]:
simclr_model, _, _ = load_model(args, train_loader, reload_model=True)
simclr_model = simclr_model.to(args.device)
simclr_model.eval()

In [ ]:
## Logistic Regression
n_classes = 5 # stl-10
model = LogisticRegression(simclr_model.n_features, n_classes)
model = model.to(args.device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
best_acc = 0.91
for epoch in range(args.logistic_epochs):
    loss_epoch, accuracy_epoch = train(args, train_loader, simclr_model, model, criterion, optimizer)
    print(f"Epoch [{epoch}/{args.logistic_epochs}]\t Loss: {loss_epoch / len(train_loader)}\t Accuracy: {accuracy_epoch / len(train_loader)}")
    if accuracy_epoch / len(train_loader) > best_acc :  
        print(f"Model {train_loader}_{accuracy_epoch / len(train_loader)} saved")
        save_model(args, simclr_model, optimizer)

# final testing
loss_epoch, accuracy_epoch = test(args, valid_loader, simclr_model, model, criterion, optimizer)
print(f"[FINAL]\t Loss: {loss_epoch / len(valid_loader)}\t Accuracy: {accuracy_epoch / len(valid_loader)}")

In [38]:
oss_epoch, accuracy_epoch = test(args, valid_loader, simclr_model, model, criterion, optimizer)
print(f"[FINAL]\t Loss: {loss_epoch / len(valid_loader)}\t Accuracy: {accuracy_epoch / len(valid_loader)}")

[FINAL]	 Loss: 1.3014336422913604	 Accuracy: 0.8099747474747475


In [41]:
oss_epoch, accuracy_epoch = test(args, valid_loader, simclr_model, model, criterion, optimizer)
print(f"[FINAL]\t Loss: {loss_epoch / len(valid_loader)}\t Accuracy: {accuracy_epoch / len(valid_loader)}")

[FINAL]	 Loss: 2.2546271115541456	 Accuracy: 0.7482666666666666


In [39]:
for epoch in range(args.logistic_epochs):
    loss_epoch, accuracy_epoch = train(args, train_loader, simclr_model, model, criterion, optimizer)
    print(f"Epoch [{epoch}/{args.logistic_epochs}]\t Loss: {loss_epoch / len(train_loader)}\t Accuracy: {accuracy_epoch / len(train_loader)}")

# final testing
loss_epoch, accuracy_epoch = test(args, test_loader, simclr_model, model, criterion, optimizer)
print(f"[FINAL]\t Loss: {loss_epoch / len(test_loader)}\t Accuracy: {accuracy_epoch / len(test_loader)}")

Step [0/37]	 Loss: 1.8033009767532349	 Accuracy: 0.136
Epoch [0/500]	 Loss: 1.3058979640135895	 Accuracy: 0.5074594594594597
Step [0/37]	 Loss: 1.1310392618179321	 Accuracy: 0.56
Epoch [1/500]	 Loss: 1.1025966373649803	 Accuracy: 0.5943783783783785
Step [0/37]	 Loss: 1.1083776950836182	 Accuracy: 0.608
Epoch [2/500]	 Loss: 1.0608275178316477	 Accuracy: 0.6023783783783784
Step [0/37]	 Loss: 0.9856696724891663	 Accuracy: 0.672
Epoch [3/500]	 Loss: 1.0526621744439408	 Accuracy: 0.6019459459459459
Step [0/37]	 Loss: 1.1596343517303467	 Accuracy: 0.56
Epoch [4/500]	 Loss: 1.0424807184451335	 Accuracy: 0.6056216216216217
Step [0/37]	 Loss: 1.0736558437347412	 Accuracy: 0.576
Epoch [5/500]	 Loss: 1.0309529691129118	 Accuracy: 0.6099459459459459
Step [0/37]	 Loss: 1.16108238697052	 Accuracy: 0.544
Epoch [6/500]	 Loss: 1.0287092763024408	 Accuracy: 0.6116756756756757
Step [0/37]	 Loss: 0.9434301853179932	 Accuracy: 0.688
Epoch [7/500]	 Loss: 1.0198058888718888	 Accuracy: 0.6147027027027027
Step

KeyboardInterrupt: 